In [1]:
import pandas as pd
import os

from functools import reduce
from glob import glob
from collections import deque

In [2]:
pwd = os.getcwd()
gp_path = pwd.split('EDA')[0][:-1]
raw_path = glob(f'{gp_path}\\**\\merge_dataset\\raw*.csv', recursive=True)[0]
tag_path = glob(f'{gp_path}\\**\\merge_dataset\\tag*.csv', recursive=True)[0]

print(raw_path, tag_path, sep='\n\n')

c:\Users\reot1\OneDrive\바탕 화면\ml2\web_crawling\merge_dataset\raw_merge.csv

c:\Users\reot1\OneDrive\바탕 화면\ml2\web_crawling\merge_dataset\tag_merge.csv


In [3]:
raw = pd.read_csv(raw_path, sep='\t')

In [4]:
raw['genre'].unique()

array(['Indie, Simulation, Strategy', 'Strategy',
       'Action, Adventure, Indie', ...,
       'Adventure, Casual, Massively Multiplayer, RPG, Simulation, Strategy',
       'Action, Adventure, Free to Play, Indie, Massively Multiplayer, Racing, RPG, Early Access',
       'Action, Massively Multiplayer, RPG, Simulation, Early Access'],
      dtype=object)

In [5]:
genre_set = reduce(lambda x,y : x|y, raw['genre'].apply(lambda x : set(str(x).replace(' ', '').split(','))))

In [6]:
len(genre_set)

genre_dict = {genre : deque([0 for i in range(len(raw))])for genre in genre_set}

In [7]:
raw['genre_set'] = raw['genre'].apply(lambda x : set(str(x).replace(' ', '').split(',')))

In [8]:
for idx in range(len(raw)):
    for genre in raw.loc[idx,'genre_set']:
        genre_dict[genre][idx] = 1

In [9]:
genre_dict['appid'] = raw['appid']

In [10]:
genre_df = pd.DataFrame(genre_dict)
genre_df = genre_df[genre_df.columns.to_list()[-1:]+ genre_df.columns.to_list()[:-1]]

In [11]:
from sklearn.cluster import KMeans

```
genre 컬럼 TF-IDF
```

In [ ]:
from functools import reduce
b = set()
for csv in csv_list[0::2]:
    print(os.path.split(csv)[1])
    
    df = pd.read_csv(csv, sep='\t')
    set(df.loc[0,'genre'].replace(' ','').split(','))
    a = reduce(lambda x,y : x | y, df['genre'].apply(lambda x: set(str(x).replace(' ','').split(','))))
    print(reduce(lambda x,y : x | y, df['genre'].apply(lambda x: set(str(x).replace(' ','').split(',')))))
    print(len(reduce(lambda x,y : x | y, df['genre'].apply(lambda x: set(str(x).replace(' ','').split(','))))))
    b |= a

In [ ]:
len(b)

In [ ]:
df = pd.read_csv(csv_list[0], sep='\t')

In [ ]:
df.head(1).T

In [ ]:
df['tag_key'] = df['tags'].apply(lambda x:set(eval(x)) if x != '[]' else set())

In [ ]:
df.loc[:,['genre', 'tag_key']].head(5)

In [ ]:
df['tag_list'] = df['tag_key'].apply(lambda x: ', '.join(list(x)))

In [ ]:
df.head(1).T

In [ ]:
df['genre'].fillna('',inplace=True)

In [ ]:
df.loc[:,['genre', 'tag_list']].head(5)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import string

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

```
string_punctuation -> 특수문자 사전
ord() ascii 코드 반환
```

In [ ]:
# 토큰화한 각 단어들의 원형을 리스트로 반환
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 텍스트를 인풋으로 넣고 토큰화시키고 토큰화된 단언들의 원형을 리스트로 반환
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(str(text).lower().translate(remove_punct_dict)))

In [ ]:
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                            stop_words='english', ngram_range=(1,2),
                            min_df=0.05, max_df =0.85)
ftr_vect = tfidf_vect.fit_transform(df['genre'])

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# KMeans: 10
km_cluster = KMeans(n_clusters=10, max_iter=10000, random_state=0)
km_cluster.fit(ftr_vect)

# cluster 및 중심 좌표 정보
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# cluster 라벨 추가
df['cluster_label'] = cluster_label
df[['genre', 'cluster_label']].head(1)

In [ ]:
df[df['cluster_label']==0][['genre', 'tag_list']]

In [ ]:
cluster_centers = km_cluster.cluster_centers_

print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

In [ ]:
def get_cluster_details(cluster_model, cluster_data, cluster_nums, 
                        feature_names, top_n_features=10):
    
    # 핵심 단어 등 정보를 담을 사전 생성
    cluster_details = {}
    
    # word 피처 중심과의 거리 내림차순 정렬시 값들의 index 반환
    center_info = cluster_model.cluster_centers_        # 군집 중심 정보
    center_descend_ind = center_info.argsort()[:, ::-1] # 행별(군집별)로 역순 정렬
    
    # 군집별 정보 담기
    for i in range(cluster_nums):
        # 군집별 정보를 담을 데이터 초기화
        cluster_details[i] = {} # 사전 안에 사전
        
        # 각 군집에 속하는 파일명
        genre = cluster_data[cluster_data["cluster_label"] == i]["genre"]
        genre = genre.values.tolist()
        
        # 군집별 중심 정보
        top_feature_values = center_info[i, :top_n_features].tolist()

        # 군집별 핵심 단어 피처명
        top_feature_indexes = center_descend_ind[i, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]
        
        # 각 군집별 정보 사전에 담기
        cluster_details[i]["cluster"] = i                              # i번째 군집
        cluster_details[i]["top_features"] = top_features              # 군집별 핵심 단어
        cluster_details[i]["top_feature_values"] = top_feature_values  # 군집별 중심 정보
        cluster_details[i]["genre"] = genre                    # 군집 속 파일명
        
    return cluster_details

In [ ]:
# TF-IDF 객체의 전체 word 명칭
feature_names = tfidf_vect.get_feature_names_out()

# 함수 적용
cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=df, cluster_nums=10,
                                feature_names=feature_names, top_n_features=10 )

In [ ]:
cluster_details.keys()

In [ ]:
cluster_details[0]['genre']